In [2]:
import json
from fastmcp import Client

In [3]:
server_url = "http://localhost:8090/simple" 

# Invoke from MCP Server with extra header

In [24]:
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=server_url,
    headers={
        "X-Mantix-Session": "my-mantix-backend-session"
    }
)
client = Client(transport)

async with Client(transport) as client:
    params = {"data": "Hello World!"}
    result = await client.call_tool_mcp("do_something", params)
    print(result)
    

meta=None content=[TextContent(type='text', text='successfully processed Hello World! for session my-mantix-backend-session.', annotations=None, meta=None)] structuredContent={'result': 'successfully processed Hello World! for session my-mantix-backend-session.'} isError=False


# get all tools from MCP server

In [102]:
async with Client(server_url) as client:
    mcp_tools = await client.list_tools()
for tool_info in list(mcp_tools):
    print(f"{tool_info}")

name='get_catalog' title=None description=None inputSchema={'properties': {}, 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='get_form' title=None description=None inputSchema={'properties': {'form_id': {'title': 'Form Id', 'type': 'string'}}, 'required': ['form_id'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='submit_form' title=None description=None inputSchema={'properties': {'form_id': {'title': 'Form Id', 'type': 'string'}, 'data': {'title': 'Data', 'type': 'string'}}, 'required': ['form_id', 'data'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'],

# get catalog of all products from MCP server

In [103]:
transport = StreamableHttpTransport(
    url=server_url,
    headers={
        "X-Mantix-Session": "my-mantix-backend-session"
    }
)

async with Client(transport) as client:
    params = {}
    result = await client.call_tool_mcp("get_catalog", params)
    json_schema = result.content[0].text
    print(json_schema)

{"64": {"id": 64, "displayName": "FirmenSchutzbrief by ELEMENT", "tag": ["Versicherung", "Gesch\u00e4ftskundentarif", "FirmenSchutz", "Einzelversicherung", "Multiger\u00e4tetarif"], "attribute": [{"displayName": "DocumentChecker: Id Dokumentenattribut", "displayDescription": "", "value": "-300", "visible": false}, {"displayName": "DocumentChecker: Name Attribut an Vertrag", "displayDescription": "", "value": "Versicherungsschein-Nr.", "visible": false}, {"displayName": "Mindestvertragslaufzeit", "displayDescription": "", "value": "12", "visible": false}, {"displayName": "max. Laufzeit Monate", "displayDescription": "", "value": "60", "visible": false}, {"displayName": "Zahlungsart", "displayDescription": "", "value": "\u00dcberweisung", "visible": false}, {"displayName": "Zahlungsweise", "displayDescription": "", "value": "einmalig", "visible": false}]}, "3415": {"id": 3415, "displayName": "HandySchutzbrief Basis ohne Diebstahl (Risikotr\u00e4ger ELEMENT)", "tag": ["Privatkundentarif",

# invoke from MCP server with json

In [5]:
async with Client(server_url) as client:
    address = {
        "typeId": -10,
        "street": "Main Street",
        "streetNumber": "5A",
        "addition": None,
        "zipcode": "10115",
        "city": "Berlin",
        "countryCode": "DE"
    }
    params = {"data": json.dumps(address)}
    result = await client.call_tool_mcp("submit_form", params)
    print(result.content[0].text)
    # pretty = json.loads(result.content[0].text)
    # print(json.dumps(pretty, indent=2, ensure_ascii=False))


successfully processed {"typeId": -10, "street": "Main Street", "streetNumber": "5A", "addition": null, "zipcode": "10115", "city": "Berlin", "countryCode": "DE"} for session None.


# receive form definition from MCP and invoke model to submit valid form

In [16]:
# receive form
async with Client(server_url) as client:
    result = await client.call_tool_mcp("get_address_form", {})
    json_schema = result.content[0].text
    print(json_schema)

{"properties": {"typeId": {"description": "Address type. -10 = Private, -11 = Business, -12 = Billing, -13 = Delivery", "title": "Typeid", "type": "integer"}, "street": {"description": "Street (without house number)", "title": "Street", "type": "string"}, "streetNumber": {"description": "House number", "title": "Streetnumber", "type": "string"}, "addition": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "Additional info (e.g. '2nd floor')", "title": "Addition"}, "zipcode": {"description": "Postal code", "title": "Zipcode", "type": "string"}, "city": {"description": "City name", "title": "City", "type": "string"}, "countryCode": {"description": "2-char ISO country code (e.g. DE)", "maxLength": 2, "minLength": 2, "title": "Countrycode", "type": "string"}}, "required": ["typeId", "street", "streetNumber", "zipcode", "city", "countryCode"], "title": "Address", "type": "object"}


In [24]:
# generate data 
from openai import OpenAI
import os
import sys
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
load_dotenv()

model = os.getenv("STACKIT_MODEL_ID")
key = os.getenv("STACKIT_KEY")
base = os.getenv("STACKIT_BASE")

messages = [
    {"role":"system", "content": (f"You are a technical function that converts user information into this pydantic object: {json_schema}"
                                  "Ommitt missing fields, unless their values are obvious. Rather respond with incomplete objects (but valid json)"
                                  "Do not comment on the response, do not provide different options - just return the object.")}, 
    # {"role":"user", "content": f"Ich wohne in der Hauptstr. 3 in 12345 Musterstadt"},     
    # {"role":"user", "content": f"Ich wohne in der Hauptstr. 3 in Musterstadt"},     
    {"role":"user", "content": f"Ich wohne in der Hauptstr. 3, dritter Stock links, in 12345 Musterstadt"},     
]

client = OpenAI(base_url=base, api_key=key)
response = client.chat.completions.create(model=model, messages=messages)
json_address = response.choices[0].message.content
print(json_address)

{"typeId": -10, "street": "Hauptstr.", "streetNumber": "3", "addition": "dritter Stock links", "zipcode": "12345", "city": "Musterstadt", "countryCode": "DE"}


In [25]:
# submit data
async with Client(server_url) as client:
    params = {"data": json_address}
    result = await client.call_tool_mcp("submit_address_form", params)
    print(result.content[0].text)

successfully processed {"typeId": -10, "street": "Hauptstr.", "streetNumber": "3", "addition": "dritter Stock links", "zipcode": "12345", "city": "Musterstadt", "countryCode": "DE"} for session None.


# Locally retrieve raw data to create dynamic form

In [29]:
# retrieve product data from intervista backend
from httpx import AsyncClient

base = os.getenv("INTERVISTA_URL")
path = os.getenv("INTERVISTA_PATH")
user = os.getenv("INTERVISTA_USER")
pwd = os.getenv("INTERVISTA_PWD")

# id = 64
# id = 3415
id = 3424
# id = 3436
# id = 3448
# id = 3571
# id = 3610

async with AsyncClient(auth=(user, pwd)) as client:
    response = await client.get(f"{base}{id}{path}")
    raw_data = response.json()["result"][0]

print(raw_data)

{'product': {'productId': 3151, 'productVariantId': 3424, 'name': 'HandySchutzbrief Basis Check24', 'description': 'Element Check24 Basis ohne Diebstahl Jahreszahler AVB 02-2023SGon', 'versionInfo': 'Element Check24 Basis ohne Diebstahl Jahreszahler AVB 02-2023SGon', 'displayName': 'HandySchutzbrief Basis ohne Diebstahl (Risikoträger ELEMENT)', 'tooltip': None, 'state': 'Offen für den Bestand', 'typeId': -101001, 'availableFrom': 1675209600000, 'availableTo': None, 'personSpecific': False, 'providerId': 10, 'providerName': 'ELEMENT', 'providerShortName': 'EL', 'tag': ['Privatkundentarif', 'Versicherung', 'Basis', 'Einzelversicherung', 'Check24'], 'tagrenderdata': None, 'rule': ['de.schutzgarant.core.plugins.ImeiDublettenCheckPlugin', 'de.schutzgarant.core.plugins.BeitragsrechnerHandyschutzbriefPrivat', 'at.intervista.collactive.sellactive.plugin.pdf.xslt.XsltFoPrintPlugin', 'de.schutzgarant.core.plugins.GeraetePlausiPlugin', 'de.schutzgarant.core.plugins.WartezeitenPlugin', 'de.schutzg

In [31]:
from pydantic import (
    Field,
    constr,
    create_model
)
from typing import Optional, Literal

# create function to retrieve attributes
def _extract_attribute_fields(model_name: str, raw_data: str):
    fields = {}
    attributes = [
        attr for cat in raw_data["attributecategorie"] for attr in cat["attribute"]
    ]

    for attr in attributes:
        if not (attr.get("inputField") is True and attr.get("hide") is False):
            continue

        # Valid Python field name
        field_name = f'attribute_{attr["attributeId"]}'

        title = attr.get("displayName", "")
        description = ""
        attr_type = attr.get("type")
        required = attr.get("required", False)
        default = attr.get("value", None)

        # Choose field type and constraints
        if attr_type == "Liste":
            allowed = tuple(val["name"] for val in attr.get("listelement", []))
            field_type = Literal[allowed] if allowed else str
            description += f" (allowed: {', '.join(allowed)})"
        elif attr_type == "Zahl":
            field_type = int
            try:
                if default is not None:
                    default = int(default)
            except:
                pass
            description += " (must be integer)"
        elif attr_type == "Datum":
            field_type = constr(pattern=r"^\d{4}-\d{2}-\d{2}$")
            description += " (YYYY-MM-DD)"
        elif attr_type == "Text":
            field_type = str
        elif attr_type == "Auswahl":
            field_type = bool
            try:
                if default is not None and isinstance(default, str):
                    default = default.lower() == "true"
            except:
                pass
        else:
            field_type = str

        if not required:
            field_type = Optional[field_type]
            # Default to None if not set
            default = default if default is not None else None
        else:
            default = default if default is not None else ...

        fields[field_name] = (
            field_type,
            Field(title=title, description=description, default=default),
        )
    return create_model(model_name, **fields)

In [39]:
Attributes = _extract_attribute_fields("Attributes", raw_data)
print(Attributes.model_json_schema())

[{'name': 'Allgemeine Kategorie', 'displayName': 'Allgemeine Kategorie', 'rank': 30, 'expandable': False, 'collapse': None, 'header': '', 'footer': '', 'displayAttributeCategoryDtoMap': {'Diebstahl': {'collapse': None, 'header': '', 'footer': ''}, 'SelfActive Erfassung': {'collapse': None, 'header': '', 'footer': ''}, 'Geräteimport': {'collapse': None, 'header': '', 'footer': ''}, 'Provisionierung': {'collapse': None, 'header': '', 'footer': ''}, 'Vertrag': {'collapse': None, 'header': '', 'footer': ''}, 'Vertragsvorlage': {'collapse': None, 'header': '', 'footer': ''}, 'Beitragsberechnung': {'collapse': None, 'header': '', 'footer': ''}, 'TouchActive Extra': {'collapse': None, 'header': '', 'footer': ''}, 'Schaden': {'collapse': None, 'header': '', 'footer': ''}, 'Vermittlermerkmale': {'collapse': None, 'header': '', 'footer': ''}, 'Eingabedaten': {'collapse': None, 'header': '', 'footer': ''}, 'Dokumentgenerierung': {'collapse': None, 'header': '', 'footer': ''}, 'TouchActive Erfassu

In [44]:
module_data = {}
module_data["attributecategorie"] = []
for sub_prod in raw_data["subproduct"]:
    ppv_id = sub_prod["partProductVariantId"]
    if not ppv_id in [19, 3112, 3091]:
        module_data["attributecategorie"].extend(
            [
                cat for cat in sub_prod["productTreeRenderData"][
                "attributecategorie"
                ]
            ]
        )
Modules = _extract_attribute_fields("Modules", module_data)
print(Modules.model_json_schema())

{'properties': {'attribute_604': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': False, 'description': '', 'title': 'Ich kenne meine IMEI-/Seriennummer noch nicht'}, 'attribute_67': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': '', 'description': '', 'title': 'IMEI-/Seriennummer'}, 'attribute_3730': {'default': '501 - 750 €', 'description': ' (allowed: 0 - 250 €, 251 - 500 €, 501 - 750 €, 751 - 1.000 €, 1.001 - 1.250 €, 1.251 - 1.500 €, 1.501 - 1.750 €, 1.751 - 2.000 €, 2.001 - 2.500 €, 2.501 - 3.000 €)', 'enum': ['0 - 250 €', '251 - 500 €', '501 - 750 €', '751 - 1.000 €', '1.001 - 1.250 €', '1.251 - 1.500 €', '1.501 - 1.750 €', '1.751 - 2.000 €', '2.001 - 2.500 €', '2.501 - 3.000 €'], 'title': 'Unsubventionierter Kaufpreis', 'type': 'string'}, 'attribute_3733': {'default': 'bis 2 Wochen', 'description': ' (allowed: bis 2 Wochen, 2 Wochen bis 6 Wochen, 6 Wochen bis halbes Jahr, halbes Jahr bis 1 Jahr, 1 Jahr bis 2 Jahre, älter als 2 Jahre)', 'enum': ['bis 2

In [70]:
form_fields = {}
form_fields["Attributes"] = (_extract_attribute_fields("Attributes", raw_data) ,Field("Attributes"))
form_fields["Modules"] = (_extract_attribute_fields("Modules", module_data) ,Field("Modules"))

Form = create_model("Form", **form_fields)
print(json_schema)

{'$defs': {'Attributes': {'properties': {'attribute_331': {'default': 'Handy/Smartphone', 'description': ' (allowed: Handy/Smartphone, Tablet, Smartwatch)', 'enum': ['Handy/Smartphone', 'Tablet', 'Smartwatch'], 'title': 'Gerätetyp', 'type': 'string'}}, 'title': 'Attributes', 'type': 'object'}, 'Modules': {'properties': {'attribute_604': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': False, 'description': '', 'title': 'Ich kenne meine IMEI-/Seriennummer noch nicht'}, 'attribute_67': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': '', 'description': '', 'title': 'IMEI-/Seriennummer'}, 'attribute_3730': {'default': '501 - 750 €', 'description': ' (allowed: 0 - 250 €, 251 - 500 €, 501 - 750 €, 751 - 1.000 €, 1.001 - 1.250 €, 1.251 - 1.500 €, 1.501 - 1.750 €, 1.751 - 2.000 €, 2.001 - 2.500 €, 2.501 - 3.000 €)', 'enum': ['0 - 250 €', '251 - 500 €', '501 - 750 €', '751 - 1.000 €', '1.001 - 1.250 €', '1.251 - 1.500 €', '1.501 - 1.750 €', '1.751 - 2.000 €', '2.001 - 

In [47]:
from pydantic import BaseModel

class Form(BaseModel):
    attributes: Attributes = Field(...)
    modules: Modules = Field(...)

json_schema = Form.model_json_schema()
print(json_schema)

{'$defs': {'Attributes': {'properties': {'attribute_331': {'default': 'Handy/Smartphone', 'description': ' (allowed: Handy/Smartphone, Tablet, Smartwatch)', 'enum': ['Handy/Smartphone', 'Tablet', 'Smartwatch'], 'title': 'Gerätetyp', 'type': 'string'}}, 'title': 'Attributes', 'type': 'object'}, 'Modules': {'properties': {'attribute_604': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': False, 'description': '', 'title': 'Ich kenne meine IMEI-/Seriennummer noch nicht'}, 'attribute_67': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': '', 'description': '', 'title': 'IMEI-/Seriennummer'}, 'attribute_3730': {'default': '501 - 750 €', 'description': ' (allowed: 0 - 250 €, 251 - 500 €, 501 - 750 €, 751 - 1.000 €, 1.001 - 1.250 €, 1.251 - 1.500 €, 1.501 - 1.750 €, 1.751 - 2.000 €, 2.001 - 2.500 €, 2.501 - 3.000 €)', 'enum': ['0 - 250 €', '251 - 500 €', '501 - 750 €', '751 - 1.000 €', '1.001 - 1.250 €', '1.251 - 1.500 €', '1.501 - 1.750 €', '1.751 - 2.000 €', '2.001 - 

# Locally fill dynamic form

In [58]:
# generate data 
from openai import OpenAI
import os
import sys
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
load_dotenv()

model = os.getenv("STACKIT_MODEL_ID")
key = os.getenv("STACKIT_KEY")
base = os.getenv("STACKIT_BASE")

messages = [
    {"role":"system", "content": (f"You are a technical function that converts user information into this pydantic object: {json_schema}"
                                  "Ommitt missing fields, unless their values are obvious. Rather respond with incomplete objects (but valid json)"
                                  "Do not comment on the response, do not provide different options - just return the object.")}, 
    # {"role":"user", "content": f"Ich hab gestern für 650€ ein neues Samsung Handy mit 12 Monaten Garantie gekauft. Die IMEI ist 123456789012345."},     
    {"role":"user", "content": f"Ich hab gestern ein neues Handy gekauft. Die IMEI ist 123456789012345."}, 
]

client = OpenAI(base_url=base, api_key=key)
response = client.chat.completions.create(model=model, messages=messages)
json_form = response.choices[0].message.content
print(json_form)

{"attributes": {"attribute_331": "Handy/Smartphone"}, "modules": {"attribute_67": "123456789012345"}}


# Locally submit dynamic form

In [61]:

def _resolve_ref(schema, ref):
    """Given a schema and a JSON pointer, resolve and return the referenced schema."""
    parts = ref.lstrip('#/').split('/')
    target = schema
    for part in parts:
        target = target[part]
    return target

def _get_schema_for_loc(schema, loc):
    """
    Traverse schema following the err['loc'], resolving refs as needed.
    Returns the field schema dict (where 'description', 'title', etc. live).
    """
    node = schema
    for i, part in enumerate(loc):
        # If $ref exists at this level, resolve it before going further
        if "$ref" in node.get("properties", {}).get(part, {}):
            node = _resolve_ref(schema, node["properties"][part]["$ref"])
        elif i < len(loc) - 1:
            # Go to next level down (should be an object property)
            node = node["properties"][part]
            # If this node is a $ref (and not directly on .properties), resolve
            if "$ref" in node:
                node = _resolve_ref(schema, node["$ref"])
        else:
            # Last part is the field itself
            node = node["properties"].get(part, {})
    return node


def _extract_field_info(schema, loc):
    """Returns title, description, enum for the field at loc in schema."""
    field_schema = _get_schema_for_loc(schema, loc)
    return (
        field_schema.get("title", loc[-1]),
        field_schema.get("description", ""),
        field_schema.get("enum", [])
    )

In [66]:
from pydantic import ValidationError

try: 
    form = Form(**json.loads(json_form))
    print(f"successfully processed {form.model_dump_json()}")
except ValidationError as e:
    schema = Form.model_json_schema()
    suggestions = []
    for err in e.errors():
        loc = err["loc"]
        title, description, choices = _extract_field_info(schema, loc)
        suggestion = f"Please provide a value for '{loc[-1]} ({title})'"
        if description:
            suggestion += f" {description}"
        if choices:
            suggestion += f". Possible values: {', '.join(choices)}"
        suggestions.append(suggestion)
    print("submission failed:\n" + "\n".join(suggestions) + f"\nThis is the full schema: {schema}") 

submission failed:
Please provide a value for 'attribute_70 (Hersteller)'  (allowed: Samsung, Apple, Huawei, Xiaomi, Acer, AEG, Alcatel, Allview, Amica, Asus, ATAG, Bauknecht, BEKO, Blackberry, Blomberg, Bomann, Bosch (BSH), Candy, Canon, CAT, Caterpillar, Clatronic, Constructa, Cubot, DELL, Doro, Electrolux, Elektra Bregenz, Epson, Exquisit, Fagor, Fairphone, Fujifilm, Fujitsu, Gaggenau, Gigaset, Google, GoPro, Gorenje, Grundig, Haier, Hewlett Packard/HP, Honor, Hoover, HTC, Ignis, Ikea, Indesit, ISY, Juno, Koenic, Körting, Küppersbusch, Leica, Lenovo, LG, Liebherr, Microsoft, Miele, Motorola, Neff, Nikon, Nokia, ok, Olympus, OnePlus, OPPO, Oranier, Panasonic, PEAQ, Privileg, Progress, Schaub Lorenz, Sharp, Sibir, Siemens, Smeg, Sony, Teka, Termikel, UleFone, Vivo, Whirlpool, Wiko, Zanker, Zanussi, ZTE, Sonstige). Possible values: Samsung, Apple, Huawei, Xiaomi, Acer, AEG, Alcatel, Allview, Amica, Asus, ATAG, Bauknecht, BEKO, Blackberry, Blomberg, Bomann, Bosch (BSH), Candy, Canon, CA

# get dynamic schema from MCP and submit compliant form

In [99]:
from fastmcp.client.transports import StreamableHttpTransport

# id = 64
# id = 3415
id = 3424
# id = 3436
# id = 3448
# id = 3571
# id = 3610

transport = StreamableHttpTransport(
    url=server_url,
    headers={
        "X-Mantix-Session": "my-mantix-backend-session"
    }
)

async with Client(transport) as client:
    params = {"form_id": str(id)}
    result = await client.call_tool_mcp("get_form", params)
    json_schema = result.content[0].text
    print(json_schema)

{"$defs": {"Address": {"properties": {"typeId": {"description": "Address type. -10 = Private, -11 = Business, -12 = Billing, -13 = Delivery", "title": "Typeid", "type": "integer"}, "street": {"description": "Street (without house number)", "title": "Street", "type": "string"}, "streetNumber": {"description": "House number", "title": "Streetnumber", "type": "string"}, "addition": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "Additional info (e.g. '2nd floor')", "title": "Addition"}, "zipcode": {"description": "Postal code", "title": "Zipcode", "type": "string"}, "city": {"description": "City name", "title": "City", "type": "string"}, "countryCode": {"description": "2-char ISO country code (e.g. DE)", "maxLength": 2, "minLength": 2, "title": "Countrycode", "type": "string"}}, "required": ["typeId", "street", "streetNumber", "zipcode", "city", "countryCode"], "title": "Address", "type": "object"}, "Attributes": {"properties": {"attribute_331": {"defaul

In [100]:
# generate data 
from openai import OpenAI
import os
import sys
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
load_dotenv()

model = os.getenv("STACKIT_MODEL_ID")
key = os.getenv("STACKIT_KEY")
base = os.getenv("STACKIT_BASE")

messages = [
    {"role":"system", "content": (f"You are a technical function that converts user information into this pydantic object: {json_schema}"
                                  "Ommitt missing fields, unless their values are obvious. Rather respond with incomplete objects (but valid json)"
                                  "Do not comment on the response, do not provide different options - just return the object.")}, 
    {"role":"user", "content": f"Ich hab gestern für 650€ ein neues Samsung Handy mit 12 Monaten Garantie gekauft. Die IMEI ist 123456789012345."},     
    # {"role":"assistant", "content": '{"Modules": {"attribute_67": "123456789012345", "attribute_70": "Samsung", "attribute_16": 12, "attribute_3730": "501 - 750 €"}'},     
    # {"role":"tool", "content": "submission failed: Please provide a value for 'Attributes (Attributes)' This is the full schema: {'$defs': {'Attributes': {'properties': {'attribute_331': {'default': 'Handy/Smartphone', 'description': ' (allowed: Handy/Smartphone, Tablet, Smartwatch)', 'enum': ['Handy/Smartphone', 'Tablet', 'Smartwatch'], 'title': 'Gerätetyp', 'type': 'string'}}, 'title': 'Attributes', 'type': 'object'}, 'Modules': {'properties': {'attribute_604': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': False, 'description': '', 'title': 'Ich kenne meine IMEI-/Seriennummer noch nicht'}, 'attribute_67': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': '', 'description': '', 'title': 'IMEI-/Seriennummer'}, 'attribute_3730': {'default': '501 - 750 €', 'description': ' (allowed: 0 - 250 €, 251 - 500 €, 501 - 750 €, 751 - 1.000 €, 1.001 - 1.250 €, 1.251 - 1.500 €, 1.501 - 1.750 €, 1.751 - 2.000 €, 2.001 - 2.500 €, 2.501 - 3.000 €)', 'enum': ['0 - 250 €', '251 - 500 €', '501 - 750 €', '751 - 1.000 €', '1.001 - 1.250 €', '1.251 - 1.500 €', '1.501 - 1.750 €', '1.751 - 2.000 €', '2.001 - 2.500 €', '2.501 - 3.000 €'], 'title': 'Unsubventionierter Kaufpreis', 'type': 'string'}, 'attribute_3733': {'default': 'bis 2 Wochen', 'description': ' (allowed: bis 2 Wochen, 2 Wochen bis 6 Wochen, 6 Wochen bis halbes Jahr, halbes Jahr bis 1 Jahr, 1 Jahr bis 2 Jahre, älter als 2 Jahre)', 'enum': ['bis 2 Wochen', '2 Wochen bis 6 Wochen', '6 Wochen bis halbes Jahr', 'halbes Jahr bis 1 Jahr', '1 Jahr bis 2 Jahre', 'älter als 2 Jahre'], 'title': 'Gerätealter', 'type': 'string'}, 'attribute_70': {'description': ' (allowed: Samsung, Apple, Huawei, Xiaomi, Acer, AEG, Alcatel, Allview, Amica, Asus, ATAG, Bauknecht, BEKO, Blackberry, Blomberg, Bomann, Bosch (BSH), Candy, Canon, CAT, Caterpillar, Clatronic, Constructa, Cubot, DELL, Doro, Electrolux, Elektra Bregenz, Epson, Exquisit, Fagor, Fairphone, Fujifilm, Fujitsu, Gaggenau, Gigaset, Google, GoPro, Gorenje, Grundig, Haier, Hewlett Packard/HP, Honor, Hoover, HTC, Ignis, Ikea, Indesit, ISY, Juno, Koenic, Körting, Küppersbusch, Leica, Lenovo, LG, Liebherr, Microsoft, Miele, Motorola, Neff, Nikon, Nokia, ok, Olympus, OnePlus, OPPO, Oranier, Panasonic, PEAQ, Privileg, Progress, Schaub Lorenz, Sharp, Sibir, Siemens, Smeg, Sony, Teka, Termikel, UleFone, Vivo, Whirlpool, Wiko, Zanker, Zanussi, ZTE, Sonstige)', 'enum': ['Samsung', 'Apple', 'Huawei', 'Xiaomi', 'Acer', 'AEG', 'Alcatel', 'Allview', 'Amica', 'Asus', 'ATAG', 'Bauknecht', 'BEKO', 'Blackberry', 'Blomberg', 'Bomann', 'Bosch (BSH)', 'Candy', 'Canon', 'CAT', 'Caterpillar', 'Clatronic', 'Constructa', 'Cubot', 'DELL', 'Doro', 'Electrolux', 'Elektra Bregenz', 'Epson', 'Exquisit', 'Fagor', 'Fairphone', 'Fujifilm', 'Fujitsu', 'Gaggenau', 'Gigaset', 'Google', 'GoPro', 'Gorenje', 'Grundig', 'Haier', 'Hewlett Packard/HP', 'Honor', 'Hoover', 'HTC', 'Ignis', 'Ikea', 'Indesit', 'ISY', 'Juno', 'Koenic', 'Körting', 'Küppersbusch', 'Leica', 'Lenovo', 'LG', 'Liebherr', 'Microsoft', 'Miele', 'Motorola', 'Neff', 'Nikon', 'Nokia', 'ok', 'Olympus', 'OnePlus', 'OPPO', 'Oranier', 'Panasonic', 'PEAQ', 'Privileg', 'Progress', 'Schaub Lorenz', 'Sharp', 'Sibir', 'Siemens', 'Smeg', 'Sony', 'Teka', 'Termikel', 'UleFone', 'Vivo', 'Whirlpool', 'Wiko', 'Zanker', 'Zanussi', 'ZTE', 'Sonstige'], 'title': 'Hersteller', 'type': 'string'}, 'attribute_217': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': '', 'description': '', 'title': 'Herstellername'}, 'attribute_130': {'default': '', 'description': '', 'title': 'Modell', 'type': 'string'}, 'attribute_16': {'default': 24, 'description': ' (must be integer)', 'title': 'Herstellergarantie (Monate)', 'type': 'integer'}}, 'required': ['attribute_70'], 'title': 'Modules', 'type': 'object'}}, 'properties': {'Attributes': {'$ref': '#/$defs/Attributes'}, 'Modules': {'$ref': '#/$defs/Modules'}}, 'required': ['Attributes', 'Modules'], 'title': 'Form_3424', 'type': 'object'}"},     
]

client = OpenAI(base_url=base, api_key=key)
response = client.chat.completions.create(model=model, messages=messages)
json_form = response.choices[0].message.content
print(json_form)

```
{
  "Modules": {
    "attribute_70": "Samsung",
    "attribute_16": 12,
    "attribute_67": "123456789012345",
    "attribute_3730": "501 - 750 €"
  }
}
```


In [101]:
async with Client(transport) as client:
    params = {"form_id": str(id), "data":json_form}
    result = await client.call_tool_mcp("submit_form", params)
    json_schema = result.content[0].text
    print(json_schema)

submission failed:
Please provide a value for 'Person (Person)'
Please provide a value for 'Payment (PaymentProvider)'
Please provide a value for 'Attributes (Attributes)'
This is the full schema: {'$defs': {'Address': {'properties': {'typeId': {'description': 'Address type. -10 = Private, -11 = Business, -12 = Billing, -13 = Delivery', 'title': 'Typeid', 'type': 'integer'}, 'street': {'description': 'Street (without house number)', 'title': 'Street', 'type': 'string'}, 'streetNumber': {'description': 'House number', 'title': 'Streetnumber', 'type': 'string'}, 'addition': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Additional info (e.g. '2nd floor')", 'title': 'Addition'}, 'zipcode': {'description': 'Postal code', 'title': 'Zipcode', 'type': 'string'}, 'city': {'description': 'City name', 'title': 'City', 'type': 'string'}, 'countryCode': {'description': '2-char ISO country code (e.g. DE)', 'maxLength': 2, 'minLength': 2, 'title': 'Countrycode', '

In [22]:
import httpx
import json

url = "http://localhost:8005/stateful_mcp/"  # Adjust to your MCP server endpoint

# The official MCP initialization request
payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2025-06-18",
        "capabilities": {},
        "clientInfo": {"name": "httpx", "version": "0.27"}
    }
}

headers = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(payload), headers=headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)
    
    # ✅ Retrieve the session id from the headers!
    session_id = resp.headers.get("mcp-session-id")
    print("Established MCP Session ID:", session_id)

tool_payload = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "select_product",
        "input": {"product_id": 17}
    }
}

tool_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json, text/event-stream",
    "mcp-session-id": session_id   # <- The session token you received!
}

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(tool_payload), headers=tool_headers)
    print("Tool call result:", resp.text)   

Status code: 200
Response headers: Headers({'date': 'Fri, 25 Jul 2025 16:25:16 GMT', 'server': 'uvicorn', 'cache-control': 'no-cache, no-transform', 'connection': 'keep-alive', 'content-type': 'text/event-stream', 'mcp-session-id': '27c73a666a6447fbbf03e6282b6e4e99', 'x-accel-buffering': 'no', 'transfer-encoding': 'chunked'})
Response body: event: message
data: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2025-06-18","capabilities":{"experimental":{},"prompts":{"listChanged":true},"resources":{"subscribe":false,"listChanged":true},"tools":{"listChanged":true}},"serverInfo":{"name":"statefull-MCP","version":"1.11.0"}}}


Established MCP Session ID: 27c73a666a6447fbbf03e6282b6e4e99
Tool call result: event: message
data: {"jsonrpc":"2.0","id":2,"error":{"code":-32602,"message":"Invalid request parameters","data":""}}




In [95]:
import httpx
import json
import time

url = "http://localhost:8005/stateful_mcp/"  # Adjust to your MCP server endpoint

# The official MCP initialization request
payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2025-06-18",
        "capabilities": {},
        "clientInfo": {"name": "httpx", "version": "0.27"}
    }
}

initialized_payload = {
    "jsonrpc": "2.0",
    "method": "notifications/initialized",
    "params": {
        "capabilities": {}
    }
}

list_tools_payload = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list",
    "params": {}   # no params needed
}

tool_payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
        "name": "secure_operation",
        "arguments": {"data": "abc"}
    }
}

tool2_payload = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "reset",
        "input": {}
    }
}

headers = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
session_id = 0

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(payload), headers=headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)
    
    # ✅ Retrieve the session id from the headers!
    session_id = resp.headers.get("mcp-session-id")
    print("Established MCP Session ID:", session_id)



    tool_headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream",
        "mcp-session-id": session_id   # <- The session token you received!
    }
    resp = client.post(url, data=json.dumps(initialized_payload), headers=tool_headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)


with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(list_tools_payload), headers=tool_headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(tool_payload), headers=tool_headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(tool2_payload), headers=tool_headers)
    print("Status code:", resp.status_code)
    print("Response headers:", resp.headers)
    print("Response body:", resp.text)
    

Status code: 200
Response headers: Headers({'date': 'Fri, 25 Jul 2025 18:33:50 GMT', 'server': 'uvicorn', 'cache-control': 'no-cache, no-transform', 'connection': 'keep-alive', 'content-type': 'text/event-stream', 'mcp-session-id': '973eccae5d6d42adaa25be35498df84a', 'x-accel-buffering': 'no', 'transfer-encoding': 'chunked'})
Response body: event: message
data: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2025-06-18","capabilities":{"experimental":{},"prompts":{"listChanged":true},"resources":{"subscribe":false,"listChanged":true},"tools":{"listChanged":true}},"serverInfo":{"name":"statefull-MCP","version":"1.11.0"}}}


Established MCP Session ID: 973eccae5d6d42adaa25be35498df84a
Status code: 202
Response headers: Headers({'date': 'Fri, 25 Jul 2025 18:33:50 GMT', 'server': 'uvicorn', 'content-type': 'application/json', 'mcp-session-id': '973eccae5d6d42adaa25be35498df84a', 'content-length': '0'})
Response body: 
Status code: 200
Response headers: Headers({'date': 'Fri, 25 Jul 20

In [33]:
tool_payload = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "select_product",
        "input": {"product_id": 17}
    }
}

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json, text/event-stream",
    "mcp-session-id": session_id   # <- The session token you received!
}

with httpx.Client() as client:
    resp = client.post(url, data=json.dumps(tool_payload), headers=headers)
    print("Tool call result:", resp.text)   

Tool call result: event: message
data: {"jsonrpc":"2.0","id":2,"error":{"code":-32602,"message":"Invalid request parameters","data":""}}




In [ ]:
async with streamablehttp_client("http://localhost:8005/stateful_mcp") as (read, write, session_id):
    async with ClientSession(read, write) as session:
        print(f"Session ID: {session_id()}")
        await session.initialize()
        print(f"Session ID: {session_id()}")
        tools_result = await session.list_tools()
        print("Available tools:", [tool.name for tool in tools_result.tools])
        print(f"Session ID: {session_id()}")
        result = await session.call_tool("select_product", {"product_id": "1234"})
        print(f"Session ID: {session_id()}")
        id = session_id()

In [23]:
mcp_tools = []
async with Client(server_url) as client:
    mcp_tools = await client.list_tools()
    print(f"Session ID: {client.session_id}")
    for tool_info in list(mcp_tools):
        print(f"{tool_info}")


AttributeError: 'Client' object has no attribute 'session_id'

In [22]:
async with Client(server_url) as client:
    params = {"product_id": 17}
    result = await client.call_tool_mcp("select_product", params)
    # print(f"Session: {client.session}")
    # pretty = json.loads(result.content[0].text)
    # print(json.dumps(pretty, indent=2, ensure_ascii=False))
    # params = {"product_id": 18}
    # result = await client.call_tool_mcp("select_product", params)


AttributeError: 'Client' object has no attribute 'initialize'

In [19]:
async with Client(server_url) as client:
    
    params = {"product_id": 17}
    result = await client.call_tool_mcp("select_product", params)
    session = client.session
    print(f"Session: {client.session}")
    pretty = json.loads(result.content[0].text)
    print(json.dumps(pretty, indent=2, ensure_ascii=False))

AttributeError: property 'session' of 'Client' object has no setter

# Execute Tool Call

In [13]:
async with Client(server_url) as client:

    params = {"customer": "Schutzgarant"}
session = client.session
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
async with Client(server_url) as client:

    params = {"customer": "Schutzgarant", "displayName":"FirmenSchutzbrief by ELEMENT"}
    result = await client.call_tool_mcp("get_form", params)
    # print(result)
    pretty = json.loads(result.content[0].text)
    print(json.dumps(pretty, indent=2, ensure_ascii=False))

{
  "voNumber": "C001006389",
  "productVariantId": 64,
  "contractCode": "Mantix",
  "desiredActivationDate": "2025-07-24",
  "payment": {
    "typeId": -5901,
    "provider": {
      "__type": "object",
      "__properties": {
        "providerType": {
          "__type": "integer",
          "__enum": [
            -160,
            -161,
            -162,
            -163,
            -164,
            -165
          ],
          "__description": "Type of payment service provider. Possible values: -160 = Credit card, -161 = Bank, -162 = Amazon Pay, -163 = PayPal, -165 = Google Pay"
        },
        "providerNumber": {
          "__type": "string",
          "__description": "e.g. BIC"
        },
        "providerName": {
          "__type": "string",
          "__description": "Name of bank, VISA, AMEX, ..."
        },
        "accountNumber": {
          "__type": "string",
          "__description": "IBAN or credit card number"
        },
        "expirationDate": {
          "

In [36]:
from openai import OpenAI


def parse_json(data):
    model = os.getenv("STACKIT_MODEL_ID")
    key = os.getenv("STACKIT_KEY")
    base = os.getenv("STACKIT_BASE")

    j_object = None
    if not data or len(data) < 5:
        return None

    js = data
    for attempt in range(3):
        # Find JSON braces
        s = js.find("{")
        e = js.rfind("}") + 1
        if s == -1 or e <= s:
            print(f"Could not find valid JSON braces in: {js}")
            js = data  # reset for LLM
        else:
            try:
                j_object = json.loads(js[s:e])
                return j_object
            except json.JSONDecodeError:
                print(f"Attempt {attempt+1}: JSON decode failed.")

        # Try LLM repair
        try:
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a JSON expert function that accepts malformed and broken JSON strings. "
                        "You reply with the corrected version that requires the least changes and fully retains the content and structure. "
                        "Do not comment or explain, just return the corrected JSON."
                    ),
                },
                {"role": "user", "content": js},
            ]
            client = OpenAI(base_url=base, api_key=key)
            response = client.chat.completions.create(model=model, messages=messages)
            js = response.choices[0].message.content
        except Exception as e:
            print(f"An error occurred with LLM: {str(e)}")
            continue

    print(
        f"Failed to parse/fix JSON after 3 attempts. Original: {data}, Last attempt: {js}"
    )
    return None

In [38]:
test_data = '{"hello":"world"}'
test_data = 'abc{"hello":"world"}'
test_data = 'abc{"hello"::"world"}'
test_data = 'abc{"hello"::"world"'


result = parse_json(test_data)

print(result)

Could not find valid JSON braces in: abc{"hello"::"world"
{'hello': 'world'}
